In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from vec3 import Vec3
import meshplot as mp
import torch
torch.manual_seed(125)
import random
random.seed(125)
import torch_f as torch_f

In [ ]:
class Node:
    """
    Class Node
    """
    def __init__(self, value, radius, left = None, right = None, position = None, cl_prob= None, ce = None, mse = None, level = None, treelevel = None):
        self.left = left
        self.data = value
        self.radius = radius
        self.position = position
        self.right = right
        self.prob = cl_prob
        self.mse = mse
        self.ce = ce
        self.children = [self.left, self.right]
        self.level = level
        self.treelevel = treelevel
    
    def agregarHijo(self, children):

        if self.right is None:
            self.right = children
        elif self.left is None:
            self.left = children

        else:
            raise ValueError ("solo arbol binario ")


    def is_leaf(self):
        if self.right is None:
            return True
        else:
            return False

    def is_two_child(self):
        if self.right is not None and self.left is not None:
            return True
        else:
            return False

    def is_one_child(self):
        if self.is_two_child():
            return False
        elif self.is_leaf():
            return False
        else:
            return True

    def childs(self):
        if self.is_leaf():
            return 0
        if self.is_one_child():
            return 1
        else:
            return 2
    
    
    def traverseInorder(self, root):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorder(root.left)
            print (root.data, root.radius)
            self.traverseInorder(root.right)

    def traverseInorderwl(self, root):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderwl(root.left)
            print (root.data, root.radius, root.level, root.treelevel)
            self.traverseInorderwl(root.right)

    def get_tree_level(self, root, c):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.get_tree_level(root.left, c)
            c.append(root.level)
            self.get_tree_level(root.right, c)

    def set_tree_level(self, root, c):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.set_tree_level(root.left, c)
            root.treelevel = c
            self.set_tree_level(root.right, c)

    def traverseInorderLoss(self, root, loss):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderLoss(root.left, loss)
            loss.append(root.prob)
            self.traverseInorderLoss(root.right, loss)
            return loss

    def traverseInorderMSE(self, root, loss):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderMSE(root.left, loss)
            loss.append(root.mse)
            self.traverseInorderMSE(root.right, loss)
            return loss

    def traverseInorderCE(self, root, loss):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderCE(root.left, loss)
            loss.append(root.ce)
            self.traverseInorderCE(root.right, loss)
            return loss

    def traverseInorderChilds(self, root, l):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            self.traverseInorderChilds(root.left, l)
            l.append(root.childs())
            self.traverseInorderChilds(root.right, l)
            return l

    def preorder(self, root):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            print (root.data, root.radius)
            self.preorder(root.left)
            self.preorder(root.right)

    def cloneBinaryTree(self, root):
     
        # base case
        if root is None:
            return None
    
        # create a new node with the same data as the root node
        root_copy = Node(root.data, root.radius)
    
        # clone the left and right subtree
        root_copy.left = self.cloneBinaryTree(root.left)
        root_copy.right = self.cloneBinaryTree(root.right)
    
        # return cloned root node
        return root_copy

    def height(self, root):
    # Check if the binary tree is empty
        if root is None:
            return 0 
        # Recursively call height of each node
        leftAns = self.height(root.left)
        rightAns = self.height(root.right)
    
        # Return max(leftHeight, rightHeight) at each iteration
        return max(leftAns, rightAns) + 1

    # Print nodes at a current level
    def printCurrentLevel(self, root, level):
        if root is None:
            return
        if level == 1:
            print(root.data, end=" ")
        elif level > 1:
            self.printCurrentLevel(root.left, level-1)
            self.printCurrentLevel(root.right, level-1)

    def printLevelOrder(self, root):
        h = self.height(root)
        for i in range(1, h+1):
            self.printCurrentLevel(root, i)


    
    def count_nodes(self, root, counter):
        if   root is not None:
            self.count_nodes(root.left, counter)
            counter.append(root.data)
            self.count_nodes(root.right, counter)
            return counter

    
    def serialize(self, root):
        def post_order(root):
            if root:
                post_order(root.left)
                post_order(root.right)
                ret[0] += str(root.data)+'_'+ str(root.radius) +';'
                
            else:
                ret[0] += '#;'           

        ret = ['']
        post_order(root)
        return ret[0][:-1]  # remove last ,

    def toGraph( self, graph, index, dec, proc=True):
        
        
        radius = self.radius.cpu().detach().numpy()
        if dec:
            radius= radius[0]
        #print("posicion", self.data, radius)
        #print("right", self.right)
        
        #graph.add_nodes_from( [ (index, {'posicion': radius[0:3], 'radio': radius[3] } ) ])
        graph.add_nodes_from( [ (self.data, {'posicion': radius[0:3], 'radio': radius[3] } ) ])
        

        if self.right is not None:
            #leftIndex = self.right.toGraph( graph, index + 1, dec)#
            self.right.toGraph( graph, index + 1, dec)#
            
            #graph.add_edge( index, index + 1 )
            graph.add_edge( self.data, self.right.data )
            #if proc:
            #    nx.set_edge_attributes( graph, {(index, index+1) : {'procesada':False}})
        
        if self.left is not None:
            #retIndex = self.left.toGraph( graph, leftIndex, dec )#
            self.left.toGraph( graph, 0, dec )#

            #graph.add_edge( index, leftIndex)
            graph.add_edge( self.data, self.left.data)
            #if proc:
            #    nx.set_edge_attributes( graph, {(index, leftIndex) : {'procesada':False}})

        else:
            #return index + 1
            return

In [ ]:
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")

In [ ]:
def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')

    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)

    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'])

    return 

def traverse(root, tree):
       
        if root is not None:
            traverse(root.left, tree)
            tree.append((root.radius, root.data))
            traverse(root.right, tree)
            return tree

def traverse_2(tree1, tree2, t_l):
       
        if tree1 is not None:
            traverse_2(tree1.left, tree2.left, t_l)
            if tree2:
                t_l.append((tree1.radius, tree2.radius))
                print((tree1.radius, tree2.radius))
            else:
                t_l.append(tree1.radius)
                print((tree1.radius))
            traverse_2(tree1.right, tree2, t_l)
            return t_l
            

def traverse_conexiones(root, tree):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            traverse_conexiones(root.left, tree)
            if root.right is not None:
                tree.append((root.data, root.right.data))
            if root.left is not None:
                tree.append((root.data, root.left.data))
            traverse_conexiones(root.right, tree)
            return tree

def arbolAGrafo (nodoRaiz):
    
    conexiones = []
    lineas = traverse_conexiones(nodoRaiz, conexiones)
    tree = []
    tree = traverse(nodoRaiz, tree)

    vertices = []
    verticesCrudos = []
    for node in tree:
        vertice = node[0][0][:3]
        rad = node[0][0][-1]
        num = node[1]
        
        #vertices.append((num, {'posicion': Vec3( vertice[0], vertice[1], vertice[2]), 'radio': rad} ))
        vertices.append((len(verticesCrudos),{'posicion': Vec3( vertice[0], vertice[1], vertice[2]), 'radio': rad}))
        verticesCrudos.append(vertice)


    G = nx.Graph()
    G.add_nodes_from( vertices )
    G.add_edges_from( lineas )
    
    return G

@count_fn
def createNode(data, radius, position = None, left = None, right = None, cl_prob = None, ce = None, mse=None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, position, left, right, cl_prob, ce, mse)
 
def deserialize(data):
    if  not data:
        return 
    nodes = data.split(';')  
    #print("node",nodes[3])
    def post_order(nodes):
                
        if nodes[-1] == '#':
            nodes.pop()
            return None
        node = nodes.pop().split('_')
        data = int(node[0])
        #radius = float(node[1])
        #print("node", node)
        #breakpoint()
        radius = node[1]
        #print("radius", radius)
        rad = radius.split(",")
        rad [0] = rad[0].replace('[','')
        rad [3] = rad[3].replace(']','')
        r = []
        for value in rad:
            r.append(float(value))
        #r =[float(num) for num in radius if num.isdigit()]
        r = torch.tensor(r, device=device)
        #breakpoint()
        root = createNode(data, r)
        root.right = post_order(nodes)
        root.left = post_order(nodes)
        
        return root    
    return post_order(nodes)    


def read_tree(filename):
    #with open('./trees/' +'prof3/' +filename, "r") as f:
    with open('./trees/' +filename, "r") as f:
        byte = f.read() 
        return byte

In [ ]:
class InternalEncoder(nn.Module):
    
    def __init__(self, input_size: int, feature_size: int, hidden_size: int):
        super(InternalEncoder, self).__init__()

        #print("init")
        # Encoders atributos
        self.attribute_lin_encoder_1 = nn.Linear(input_size,feature_size)
        self.attribute_lin_encoder_2 = nn.Linear(feature_size,hidden_size)
        self.attribute_lin_encoder_3 = nn.Linear(hidden_size,feature_size)

        # Encoders derecho e izquierdo
        self.right_lin_encoder_1 = nn.Linear(feature_size,hidden_size)
        self.right_lin_encoder_2 = nn.Linear(hidden_size,feature_size)
        #self.right_lin_encoder_3 = nn.Linear(feature_size,feature_size)

        self.left_lin_encoder_1  = nn.Linear(feature_size,hidden_size)
        self.left_lin_encoder_2  = nn.Linear(hidden_size,feature_size)
        #self.left_lin_encoder_3  = nn.Linear(feature_size,feature_size)


        # Encoder final
        self.final_lin_encoder_1 = nn.Linear(2*feature_size, feature_size)

        # Funciones / Parametros utiles
        self.tanh = nn.Tanh()
        self.feature_size = feature_size


    def forward(self, input, right_input, left_input):
        # Encodeo los atributos
        attributes = self.attribute_lin_encoder_1(input)
        attributes = self.tanh(attributes)
        attributes = self.attribute_lin_encoder_2(attributes)
        attributes = self.tanh(attributes)
        attributes = self.attribute_lin_encoder_3(attributes)
        attributes = self.tanh(attributes)
        #print("attributes", attributes)

        # Encodeo el derecho
        if right_input is not None:
            #print("right input", right_input)
            context = self.right_lin_encoder_1(right_input)
            context = self.tanh(context)
            context = self.right_lin_encoder_2(context)
            #context = self.tanh(context)
            #context = self.right_lin_encoder_3(context)
            
            # Encodeo el izquierdo
            #print("left input", left_input)
            if left_input is not None:
                left = self.left_lin_encoder_1(left_input)
                #print("izquierdo", left.shape)
                left = self.tanh(left)
                #left = self.left_lin_encoder_2(left)
                #left = self.tanh(left)
                context += self.left_lin_encoder_2(left)
                #print("context izquierdo", context.shape)
        else:
            context = torch.zeros(input.shape[0],self.feature_size, requires_grad=True, device=device)
        

        context = self.tanh(context)
        feature = torch.cat((attributes,context), 1)
        feature = self.final_lin_encoder_1(feature)
        feature = self.tanh(feature)
        #print("output", feature)
        return feature

       
    

class GRASSEncoder(nn.Module):
    
    def __init__(self, input_size: int, feature_size : int, hidden_size: int):
        super(GRASSEncoder, self).__init__()
        self.leaf_encoder = InternalEncoder(input_size,feature_size, hidden_size)
        self.internal_encoder = InternalEncoder(input_size,feature_size, hidden_size)
        self.bifurcation_encoder = InternalEncoder(input_size,feature_size, hidden_size)
        
    def leafEncoder(self, node, right=None, left = None):
        return self.internal_encoder(node, right, left)
    def internalEncoder(self, node, right, left = None):
        return self.internal_encoder(node, right, left)
    def bifurcationEncoder(self, node, right, left):
        
        return self.bifurcation_encoder(node, right, left)

Grassencoder = GRASSEncoder(input_size = 4, feature_size=512, hidden_size=1024)
Grassencoder = Grassencoder.to(device)


def encode_structure_fold(fold, root):
    
    
    def encode_node(node):
        
        if node is None:
            return
        
        if node.is_leaf():
            return fold.add('leafEncoder', node.radius)
        else:
            left = encode_node(node.left)
            right = encode_node(node.right)
            if left is not None:
             
                return fold.add('bifurcationEncoder', node.radius, right, left)
            else:
                return fold.add('internalEncoder', node.radius, right)
        

    encoding = encode_node(root)
    
    return encoding
  
def encode_structure(root):
    
    def encode_node(node):
          
        if node is None:
            return
        if node.is_leaf():
            return Grassencoder.leafEncoder(node.radius.reshape(-1,4))
        else :
            left = encode_node(node.left)
            right = encode_node(node.right)
            if left is not None:
                return Grassencoder.bifurcationEncoder(node.radius.reshape(-1,4), right, left)
            else:
                return Grassencoder.internalEncoder(node.radius.reshape(-1,4), right)
        

    encoding = encode_node(root)
   
    return encodi

In [ ]:
def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return 


def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def norm(root, minx, miny, minz, minr, maxx, maxy, maxz, maxr):
    
    if root is not None:
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()
       
        root.radius[0] = (root.radius[0] - minx)/(maxx - minx)
        root.radius[1] = (root.radius[1] - miny)/(maxy - miny)
        root.radius[2] = (root.radius[2] - minz)/(maxz - minz)
        root.radius[3] = (root.radius[3] - minr)/(maxr - minr)
        
        norm(root.left, mx, my, mz, mr, Mx, My, Mz, Mr)
        norm(root.right, mx, my, mz, mr, Mx, My, Mz, Mr)
        return 

def normalize_features(root):
    features = []
    features = traversefeatures(root, features)
    
    x = [tensor[0] for tensor in features]
    y = [tensor[1] for tensor in features]
    z = [tensor[2] for tensor in features]
    r = [tensor[3] for tensor in features]
 
    norm(root, min(x), min(y), min(z), min(r), max(x), max(y), max(z), max(r))

    return 

def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features


In [ ]:
class NodeClassifier(nn.Module):
    
    def __init__(self, latent_size : int, hidden_size : int):
        super(NodeClassifier, self).__init__()
        self.mlp1 = nn.Linear(latent_size, hidden_size)
        self.mlp2 = nn.Linear(hidden_size, hidden_size)
        self.mlp3 = nn.Linear(hidden_size, 3)
        self.tanh = nn.Tanh()

    def forward(self, input_feature):
        #print("classifier input", input_feature)
        output = self.mlp1(input_feature)
        output = self.tanh(output)
        output = self.mlp2(output)
        output = self.tanh(output)
        output = self.mlp3(output)
        #print("classifier output", output)

        return output


In [ ]:
def searchNode(node, key):
     
    if (node == None):
        return False
 
    if (node.data == key):
        return node
        
 
    """ then recur on left subtree """
    res1 = searchNode(node.left, key)
    # node found, no need to look further
    if res1:
        return res1
 
    """ node is not found in left,
    so recur on right subtree """
    res2 = searchNode(node.right, key)
    return res2

def getLevelUtil(node, data, level):
    if (node == None):
        return 0
 
    if (node.data == data):
        return level
 
    downlevel = getLevelUtil(node.left, data, level + 1)

    if (downlevel != 0):
        return downlevel
 
    downlevel = getLevelUtil(node.right, data, level + 1)
    return downlevel
 
# Returns level of given data value
 
 
def getLevel(node, data):
    return getLevelUtil(node, data, 1)
 

c = []
n_nodes = input.count_nodes(input, c)
print("n_nodes", len(n_nodes))
for x in range(0, len(n_nodes)):
        level = getLevel(input, x)
        if (level):
            #print("Level of", x, "is", getLevel(input, x))
            node = searchNode(input, x)
            node.level = getLevel(input, x)
        else:
            print(x, "is not present in tree")

DATALOADER

In [ ]:
def my_collate(batch):
    return batch

#t_list = ['ArteryObjAN1-7.dat','ArteryObjAN1-0.dat', 'ArteryObjAN1-17.dat',  'ArteryObjAN1-11.dat']

#t_list = ['ArteryObjAN1-0.dat','ArteryObjAN1-7.dat', 'ArteryObjAN1-17.dat',  'ArteryObjAN1-11.dat', 'ArteryObjAN1-19.dat', 'ArteryObjAN2-4.dat', 'ArteryObjAN2-6.dat', 
#           'ArteryObjAN25-18.dat']
#t_list = ['ArteryObjAN1-17-55.dat', 'ArteryObjAN1-17-22.dat', "ArteryObjAN1-17-12.dat", "ArteryObjAN1-17-9.dat",'ArteryObjAN1-17-42.dat', 'ArteryObjAN1-17-64.dat', "ArteryObjAN1-17-70.dat", "ArteryObjAN1-17-1.dat"]
#t_list = ['ArteryObjAN1-17.dat']
#t_list = ['ArteryObjAN1-11.dat']


#t_list = ['test2.dat']

#t_list = ['ArteryObjAN31-14.dat']
#t_list = os.listdir("./trees")[:20]
t_list = os.listdir("./trees/prof4")[:20]
print(t_list)
class tDataset(Dataset):
    def __init__(self, dir, transform=None):
        self.names = dir
        self.transform = transform
        self.data = [] #lista con las strings de todos los arboles
        for file in self.names:
            self.data.append(read_tree('prof4/'+file))
        self.trees = []
        for tree in self.data:
            deserial = deserialize(tree)
            normalize_features(deserial)
            self.trees.append(deserial)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        #file = self.names[idx]
        #string = read_tree(file)
        tree = self.trees[idx]
        return tree

batch_size = 10
dataset = tDataset(t_list)
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)


In [ ]:
n_no = []
qzero = 0
qOne = 0
qtwo = 0

for batch in data_loader:
    for tree in batch:
        count = []
        n = tree.count_nodes(tree, count)
        n_no.append(len(n))
        li = []
        tree.traverseInorderChilds(tree, li)
        zero = [a for a in li if a == 0]
        one = [a for a in li if a == 1]
        two = [a for a in li if a == 2]
        qzero += len(zero)
        qOne += len(one)
        qtwo += len(two)

print(len(data_loader)*batch_size)
print(n_no)
nprom = np.mean(n_no)
print(nprom)
qzero /= len(data_loader)*batch_size
qOne /= len(data_loader)*batch_size
qtwo /= len(data_loader)*batch_size

print(qzero)
print(qOne)
print(qtwo)

In [ ]:
class Decoder(nn.Module):
    
    """ Decode an input (parent) feature into a left-child and a right-child feature """
    def __init__(self, latent_size : int, hidden_size : int):
        super(Decoder, self).__init__()
        #self.mlp = nn.Linear(latent_size,hidden_size)
        self.mlp = nn.Linear(latent_size,hidden_size)
        self.lp2 = nn.Linear(hidden_size, hidden_size)
        self.lp3 = nn.Linear(hidden_size, latent_size)

        self.mlp_left = nn.Linear(latent_size, hidden_size)
        self.mlp_left2 = nn.Linear(hidden_size, latent_size)
        #self.mlp_left3 = nn.Linear(latent_size, latent_size)
        self.mlp_right = nn.Linear(latent_size, hidden_size)
        self.mlp_right2 = nn.Linear(hidden_size, latent_size)
        #self.mlp_right3 = nn.Linear(latent_size, latent_size)


        self.mlp2 = nn.Linear(latent_size,latent_size)
        self.mlp3 = nn.Linear(latent_size,4)
        self.tanh = nn.Tanh()

    def common_branch(self, parent_feature):
        vector = self.mlp(parent_feature)
        vector = self.tanh(vector)
        vector = self.lp2(vector)
        vector = self.tanh(vector)
        vector = self.lp3(vector)
        vector = self.tanh(vector)
        return vector

    def attr_branch(self, vector):
        vector = self.mlp2(vector)
        vector = self.tanh(vector)
        vector = self.mlp3(vector)        
        return vector

    def right_branch(self, vector):
        right_feature = self.mlp_right(vector)
        right_feature = self.tanh(right_feature)
        right_feature = self.mlp_right2(right_feature)
        right_feature = self.tanh(right_feature)
        #right_feature = self.mlp_right3(right_feature)
        #right_feature = self.tanh(right_feature)
        return right_feature

    def left_branch(self, vector):
        left_feature = self.mlp_left(vector)
        left_feature = self.tanh(left_feature)
        left_feature = self.mlp_left2(left_feature)
        left_feature = self.tanh(left_feature)
        #left_feature = self.mlp_left3(left_feature)
        #left_feature = self.tanh(left_feature)
        return left_feature

    def forward(self, parent_feature):
      
        vector      = self.common_branch(parent_feature)
        attr_vector = self.attr_branch(vector)
        return attr_vector 

    def forward1(self, parent_feature):
    

        vector       = self.common_branch(parent_feature)
        attr_vector  = self.attr_branch(vector)
        right_vector = self.right_branch(vector)
        
        #print("right vector", right_vector)
        #print("radius", attr_vector)
        return right_vector, attr_vector

    def forward2(self, parent_feature):
       

        vector       = self.common_branch(parent_feature)
        attr_vector  = self.attr_branch(vector)
        right_vector = self.right_branch(vector)
        left_vector  = self.left_branch(vector)
        #print("left vector", left_vector)
        #print("right vector", right_vector)
        #print("radius", attr_vector)
        return left_vector, right_vector, attr_vector



class GRASSDecoder(nn.Module):
    def __init__(self, latent_size : int, hidden_size: int, mult: torch.Tensor):
        super(GRASSDecoder, self).__init__()
        self.decoder = Decoder(latent_size, hidden_size)
        self.node_classifier = NodeClassifier(latent_size, hidden_size)
        self.mseLoss = nn.MSELoss()  # pytorch's mean squared error loss
        self.ceLoss = nn.CrossEntropyLoss(weight = mult)  # pytorch's cross entropy loss (NOTE: no softmax is needed before)
        


    def featureDecoder(self, feature):
        return self.decoder.forward(feature)

    def internalDecoder(self, feature):
        return self.decoder.forward1(feature)

    def bifurcationDecoder(self, feature):
        return self.decoder.forward2(feature)

    def nodeClassifier(self, feature):
        return self.node_classifier(feature)

    def calcularLossAtributo(self, nodo, radio):
        #print("nodo", nodo)
        #print("radio", radio)
        a, b = list(zip(*nodo))# a son los atributos, b los pesos
        if nodo is None:
            return
        else:
            nodo = torch.stack(list(a))
        
            l = [self.mseLoss(b.reshape(1,4), gt.reshape(1,4)) for b, gt in zip(radio.reshape(-1,4), nodo.reshape(-1,4))]
            #print("mse", l)
            return l


    def classifyLossEstimator(self, label_vector, original):
        if original is None:
            return
        else:
           
            v = []
            for o in original:
                if o == 0:
                    vector = torch.tensor([1, 0, 0], device=device, dtype = torch.float)
                if o == 1:
                    vector = torch.tensor([0, 1, 0], device=device, dtype = torch.float)
                if o == 2:
                    vector = torch.tensor([0, 0, 1], device=device, dtype = torch.float)
                v.append(vector)
            

            v = torch.stack(v)
            
            l = [self.ceLoss(b.reshape(1,3), gt.reshape(1,3)).mul(0.4) for b, gt in zip(label_vector.reshape(-1,3), v.reshape(-1,3))]
         

            return l
            #return c

    '''
    def vectorAdder(self, v1, v2, v3 = None, v4 = None):
        
        print("loss estructura", v1)
        print("loss atributo", v2)
        print("right loss", v3)
        print("left loss", v4)


        v = v1.add(v2)
        #print("v0", v)
        if v3 is not None:
            v = v.add(v3)
        #print("v0", v)
        if v4 is not None:
            v = v.add(v4)
       
        return v

    '''
    def vectorAdder(self, v1, v2):
        v = v1.add(v2)
        return v

    def vectorMult(self, m, v):
        #print("v", v)
        #print("m", m)
        z = zip(v, m)
        r = []
        for c, d in z:
            #print("v", c)
            #print("m", d)
            r.append(torch.mul(c, d))
        #res = [torch.mul(v, m) for v, m in zip(v, m)]
        #print("res", r)
        return r

if round(qzero) == 0:
    qzero = 1
if round(qOne) == 0:
    qOne = 1
if round(qtwo) == 0:
    qtwo = 1
mult = torch.tensor([1/round(qzero),1/round(qOne),1/round(qtwo)], device = device)

Grassdecoder = GRASSDecoder(latent_size=512, hidden_size=1024, mult = mult)
Grassdecoder = Grassdecoder.to(device)

In [ ]:
def calcularLossEstructura(cl_p, original):
    
    if original is None:
        return
    #mult = torch.tensor([1/round(qzero),1/round(qOne),1/round(qtwo)], device = device)
    ce = nn.CrossEntropyLoss(weight = mult)

    if original.childs() == 0:
        vector = [1, 0, 0] 
    if original.childs() == 1:
        vector = [0, 1, 0]
    if original.childs() == 2:
        vector = [0, 0, 1] 


    c = ce(cl_p, torch.tensor(vector, device=device, dtype = torch.float).reshape(1, 3))
    #print("original", vector)
    #print("clasificador", cl_p)
    #print("ce", 0.4*c)
    return c


def calcularLossAtributo(nodo, radio):
    if nodo is None:
        return
    #print("nodo", nodo)
    #print("radio", radio)

    radio = radio.reshape(-1,4)
    nodo = nodo.radius.reshape(-1,4)
    l2    = nn.MSELoss()
   
    mse = l2(radio, nodo)
    #print("mse", mse)
    return mse

In [ ]:
for d in data_loader:
    for data in d:
        count = []
        numerar_nodos(data, count)
        c = []
        n_nodes = data.count_nodes(data, c)
        for x in range(0, len(n_nodes)):
            level = getLevel(data, x)
            if (level):
                #print("Level of", x, "is", getLevel(input, x))
                node = searchNode(data, x)
                node.level = getLevel(data, x)
            else:
                print(x, "is not present in tree")
        tree_level = []
        data.get_tree_level(data, tree_level)
        data.set_tree_level(data, sum(tree_level))

In [ ]:
def decode_structure_fold_grass(fold, v, root):
   
    def decode_node(fold, v, node):
        
        
        if node.childs() == 0 : 

            radio = fold.add('featureDecoder', v)
            lossAtributo = fold.add('calcularLossAtributo', node, radio)

            label = fold.add('nodeClassifier', v)
            
            lossEstructura = fold.add('classifyLossEstimator', label, node)  
            multipl = node.level/node.treelevel
            losse = fold.add('vectorMult', multipl, lossEstructura)
            
            loss =  fold.add('vectorAdder', losse, lossAtributo)       
            return loss

            
            
        elif node.childs() == 1 :
            right, radius = fold.add('internalDecoder', v).split(2)
            label = fold.add('nodeClassifier', v)
            nodoSiguiente = node.right
            if nodoSiguiente is not None:
                right_loss = decode_node(fold, right, nodoSiguiente)

            lossEstructura = fold.add('classifyLossEstimator', label, node)
            lossAtributo = fold.add('calcularLossAtributo', node, radius)
            multipl = node.level/node.treelevel
            losse = fold.add('vectorMult', multipl, lossEstructura)
            loss = fold.add('vectorAdder', losse, lossAtributo)
            
        
            loss2 = fold.add('vectorAdder', loss, right_loss)
            return loss2
            
            

        elif node.childs() == 2 :
            left, right, radius = fold.add('bifurcationDecoder', v).split(3)
            
            label = fold.add('nodeClassifier', v)            
            
            nodoSiguienteRight = node.right
            nodoSiguienteLeft = node.left


            if nodoSiguienteRight is not None:
                right_loss = decode_node(fold, right, nodoSiguienteRight)
             
            if nodoSiguienteLeft is not None:
                left_loss  = decode_node(fold, left, nodoSiguienteLeft)

            multipl = node.level/node.treelevel
            lossEstructura = fold.add('classifyLossEstimator', label, node)
            losse = fold.add('vectorMult', multipl, lossEstructura)
            lossAtributo   = fold.add('calcularLossAtributo', node, radius)
            loss = fold.add('vectorAdder', losse, lossAtributo)
            loss2 = fold.add('vectorAdder', loss, right_loss)
            loss3 = fold.add('vectorAdder', loss2, left_loss)
            
            return loss3
            

    dec = decode_node (fold, v, root)
    return dec